In [ ]:
#%tensorflow_version 1.15.3

#import tensorflow as tf
#import keras

#print(tf.__version__)
#print(tf.test.is_gpu_available())
#print(keras.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.3`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
1.15.3
False
2.2.4


In [ ]:
!pip install tensorflow==1.15.3
!pip install tensorflow-gpu==1.15.3
!pip install keras==2.2.4
!pip install git+https://github.com/matterport/Mask_RCNN.git

     |████████████████████████████████| 110.5MB 96kB/s 
     |████████████████████████████████| 3.8MB 55.6MB/s 
     |████████████████████████████████| 512kB 53.0MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=78a8f476bd87648b7713ebce89abf2e107a40e2df11eaec8aca5ff6e34144417
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [ ]:
import tensorflow as tf
import keras

print(tf.__version__)
print(keras.__version__)
tf.test.is_gpu_available()

Using TensorFlow backend.


1.15.3
2.2.4


True

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
DATA_ROOT = '/content/gdrive/My Drive/images'
RESOLUTION = 256

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN

class BuildingDataset(Dataset):

    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):

        # Read annotations and get the list of images to include in the model
        annotations = json.load(open(f'{DATA_ROOT}/image_boxes.json'))
        images = list(sorted(annotations.keys()))
        train_images, test_images = images[:int(0.8 * len(images))], images[int(0.8 * len(images)):]

        # define one class
        self.add_class("dataset", 1, "building")

        # Add images
        dataset_images = train_images if is_train else test_images
        for image in dataset_images:
            image_path = f'{DATA_ROOT}/data/256/{image}'
            image_id = image
            box = annotations[image]

            # add to dataset
            self.add_image('dataset', image_id=image_id, path=image_path, box=box)

        # Add image with no boxes
        bad_images = sorted([i for i in os.listdir(f'{DATA_ROOT}/data/256_bad') if i!='.DS_Store'])[:200]
        bad_train_images, bad_test_images = bad_images[:int(0.8 * len(bad_images))], bad_images[int(0.8 * len(bad_images)):]
        bad_dataset_images = bad_train_images if is_train else bad_test_images
        for image in bad_dataset_images:
            image_path = f'{DATA_ROOT}/data/256_bad/{image}'
            image_id = image

            # add to dataset
            self.add_image('dataset', image_id=image_id, path=image_path, box=None)
 
    # load the masks for an image
    def load_mask(self, image_id):

        # Get details of image
        info = self.image_info[image_id]
        if info['box'] is None:
            masks = np.zeros([0, 0, 0], dtype='uint8')
        else:
            box = (np.clip(np.array(info['box']), 0, 1) * 255).astype(np.uint8)
            masks = np.zeros([RESOLUTION, RESOLUTION, 1], dtype='uint8')
            # create masks
            row_1, row_2 = box[1], box[3]
            col_1, col_2 = box[0], box[2]
            masks[row_1:row_2, col_1:col_2, 0] = 1

        nb_masks = masks.shape[2]

        return masks, np.asarray([self.class_names.index('building')] * nb_masks, dtype='int32')
 

    # load an image reference
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [ ]:
# train set
train_set = BuildingDataset()
train_set.load_dataset('building', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
 
# test/val set
test_set = BuildingDataset()
test_set.load_dataset('building', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Train: 411
Test: 103


In [ ]:
# define a configuration for the model
class BuildingConfig(Config):
    # Give the configuration a recognizable name
    NAME = "building_cfg"
    # Number of classes (background + kangaroo)
    NUM_CLASSES = 1 + 1
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 125

    IMAGES_PER_GPU = 1


In [ ]:
model_path = f'/{DATA_ROOT}/mask_rcnn_coco.h5'

config = BuildingConfig()
print(config.BATCH_SIZE)
model = MaskRCNN(mode='training', model_dir=f'/{DATA_ROOT}/building_detection/', config=config)
model.load_weights(model_path, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

1






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead








In [ ]:
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=50, layers='heads')


Starting at epoch 20. LR=0.001

Checkpoint Path: //content/gdrive/My Drive/images/building_detection/building_cfg20201119T1310/mask_rcnn_building_cfg_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_co

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multi

Epoch 21/50
125/125 [==============================] - 89s 709ms/step - loss: 0.4491 - rpn_class_loss: 0.0034 - rpn_bbox_loss: 0.1534 - mrcnn_class_loss: 0.0404 - mrcnn_bbox_loss: 0.0874 - mrcnn_mask_loss: 0.1645 - val_loss: 0.5305 - val_rpn_class_loss: 0.0035 - val_rpn_bbox_loss: 0.1848 - val_mrcnn_class_loss: 0.0601 - val_mrcnn_bbox_loss: 0.1325 - val_mrcnn_mask_loss: 0.1496
Epoch 22/50
125/125 [==============================] - 48s 385ms/step - loss: 0.3887 - rpn_class_loss: 0.0030 - rpn_bbox_loss: 0.1251 - mrcnn_class_loss: 0.0420 - mrcnn_bbox_loss: 0.0655 - mrcnn_mask_loss: 0.1531 - val_loss: 0.6511 - val_rpn_class_loss: 0.0042 - val_rpn_bbox_loss: 0.2538 - val_mrcnn_class_loss: 0.0578 - val_mrcnn_bbox_loss: 0.1499 - val_mrcnn_mask_loss: 0.1854
Epoch 23/50
125/125 [==============================] - 48s 386ms/step - loss: 0.4581 - rpn_class_loss: 0.0025 - rpn_bbox_loss: 0.1494 - mrcnn_class_loss: 0.0459 - mrcnn_bbox_loss: 0.0940 - mrcnn_mask_loss: 0.1663 - val_loss: 0.6486 - val_rp